In [1]:
# Install requirements
!pip -q install langchain openai duckduckgo-search chromadb pydantic tiktoken gradio=="3.48.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.6

In [2]:
# Set API Key
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

Enter OpenAI API Key: ··········


In [3]:
# Download data
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=16bdTUIraNK4h_32LA6Qr3VqxMTMmyJkU' -O "address_book.csv"

from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="address_book.csv")
data = loader.load()

--2024-01-31 22:11:11--  https://drive.google.com/uc?export=download&id=16bdTUIraNK4h_32LA6Qr3VqxMTMmyJkU
Resolving drive.google.com (drive.google.com)... 173.194.213.138, 173.194.213.101, 173.194.213.113, ...
Connecting to drive.google.com (drive.google.com)|173.194.213.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=16bdTUIraNK4h_32LA6Qr3VqxMTMmyJkU&export=download [following]
--2024-01-31 22:11:11--  https://drive.usercontent.google.com/download?id=16bdTUIraNK4h_32LA6Qr3VqxMTMmyJkU&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.211.132, 2607:f8b0:400c:c10::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.211.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1975 (1.9K) [application/octet-stream]
Saving to: ‘address_book.csv’

address_book.csv    100%[===================>]   1.93K  --.

In [4]:
# Raw vector retrieval
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=data, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("What's Chris' phone number?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[Document(page_content='Name: Christopher Moore\nEmail: chrism@email.com\nPhone: 555-0112\nAddress: 909 Palm St\nCity: Creekview', metadata={'row': 11, 'source': 'address_book.csv'}),
 Document(page_content='Name: Ryan Lee\nEmail: ryanl@email.com\nPhone: 555-0124\nAddress: 2121 Cherry St\nCity: Highland', metadata={'row': 23, 'source': 'address_book.csv'}),
 Document(page_content='Name: Jason Harris\nEmail: jasonh@email.com\nPhone: 555-0114\nAddress: 1111 Fir St\nCity: Sunnyside', metadata={'row': 13, 'source': 'address_book.csv'}),
 Document(page_content='Name: Chloe Wilson\nEmail: chloew@email.com\nPhone: 555-0127\nAddress: 2424 Palm St\nCity: Riverside', metadata={'row': 26, 'source': 'address_book.csv'})]

In [5]:
# Set up LLM
from langchain.chat_models import ChatOpenAI

contacts_llm = llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
# Perform RAG
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

contacts_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's query."),
    ("user", "{question}"),
    ("user", "{context}")
    ])

# LangChain expression language to call our LLM using the prompt template above
contacts_chain = (
    {"question": lambda x: x["question"],
     "context": lambda x: retriever.get_relevant_documents(x["question"]),
     }
    | contacts_prompt
    | contacts_llm
    | StrOutputParser()
)

contacts_chain.invoke({"question": "Get me Chris and Madison's phone numbers"})

'Here are the phone numbers for Chris and Madison:\n\nMadison Thomas: 555-0121\nChristopher Moore: 555-0112'

In [7]:
# Create functions for our tool

from pydantic import BaseModel, Field

# Calculator function
def calculate(operation, a, b):
    if operation not in ["multiply", "divide", "add", "subtract", "exponentiate"]:
        raise ValueError(f"Invalid operation: {operation}")

    if operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Cannot divide by zero.")
        return a / b
    elif operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "exponentiate":
        return a ** b

# Define schema - helps Agent understand usage
class CalculatorTool(BaseModel):
  operation: str = Field(description="Valid inputs: 'multiply', 'divide', 'add', 'subtract', or 'exponentiate'.")
  a: float = Field(description="The first number in the operation.")
  b: float = Field(description="The second number in the operation.")

In [8]:
# Use StructuredTool for multiple inputs
from langchain.tools import StructuredTool, Tool

calculator_tool = StructuredTool.from_function(
    func=calculate,
    name="Calculator",
    description="Use for arithmetic between two terms.",
    args_schema=CalculatorTool
)

In [9]:
# Define contacts tool
contacts_tool = Tool.from_function(
    func = retriever.get_relevant_documents,
    name="Contacts",
    description="Search for contacts"
)

In [10]:
# Give our agent our calculator tool, a DuckDuckGo search tool, and our contacts search tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools.render import format_tool_to_openai_tool


lc_tools = [DuckDuckGoSearchRun(), calculator_tool, contacts_tool]
oai_tools = [format_tool_to_openai_tool(tool) for tool in lc_tools]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_tool` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [11]:
# LLM for our agent's use
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")

In [12]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful event-planning assistant. Use specific details to support your responses."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [13]:
from langchain_core.messages import AIMessage, HumanMessage

# For use in agent. From https://python.langchain.com/docs/modules/agents/how_to/custom_agent#adding-memory
chat_history = []

In [14]:
# Define our agent in LCEL
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from operator import itemgetter


agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm.bind(tools=oai_tools)
    | OpenAIToolsAgentOutputParser()
)

In [15]:
# Create an agent executor
from langchain.agents import AgentExecutor, AgentType, Tool, initialize_agent

agent_executor = AgentExecutor(agent=agent, tools=lc_tools, verbose=True)

In [16]:
def gen_response(input, _):
  response = agent_executor.invoke(
      {
          "input": input,
          "chat_history": chat_history
      }
  )["output"]

  save_to_memory(input, response)

  return response

def save_to_memory(input, response):
  chat_history.extend(
    [
        HumanMessage(content=input),
        AIMessage(content=response),
    ]
  )

In [19]:
gen_response("What's a good spot for BBQ in SF?",_)



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'best BBQ spots in San Francisco'}`




/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


Foursquare 1941 Oak Park Blvd, Pleasant Hill, CA 94523 (925) 942-0149 Visit Website Also featured in: 18 New and Classic Bay Area Barbecue Joints That Are Open for Takeout Pig In A Pickle Showing 8 of 8 Breakwater Barbecue 30 Ave Portola #1B, El Granada See on map Add to wishlist Brian Feulner / The Chronicle Brian Feulner / The Chronicle You often have to make a trek for good... Read more. With big events taking place every now and then, San Francisco attracts more than 25 million visitors annually. People usually visit San Francisco for the beautiful views and various destination spots. But you are one of those people and love BBQ too, here are our top picks for the best BBQ in San Francisco. Contents [ show] 3. Tommy's Joynt 8.2 1101 Geary Blvd (at Van Ness), San Francisco, CA BBQ Joint · Cathedral Hill · 213 tips and reviews Blake Samic: Great home-style BBQ and beer selection - and pretty cheap. Try the brisket. Mark Heyert: Specials are always good. Manna To start our list of whe

'One of the top picks for the best BBQ in San Francisco is "Tommy\'s Joynt" located at 1101 Geary Blvd (at Van Ness), San Francisco, CA. It\'s a popular BBQ joint with a great home-style BBQ and a wide beer selection. Customers recommend trying the brisket, and the specials are always good.'

In [17]:
"""
Time to try interacting with our agent! Launch in Gradio and ask it the following questions

* What's a good spot for BBQ in SF?
* What are Chris and Madison's phone numbers?
* I have 3 cakes. Each has 9 slices. If have 20 guests, how many slices will I have left over?
"""
import gradio as gr

gr.ChatInterface(
    gen_response, retry_btn=False, undo_btn=False, clear_btn=False
).queue().launch(debug=True)

Exception in thread Thread-14 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 62, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_event_loop_policy().get_event_loop()
  File "/usr/lib/python3.10/asyncio/events.py", line 656, in get_event_loop
    raise RuntimeError('There is no current event loop in thread %r.'
RuntimeError: There is no current event loop in thread 'Thread-14 (run)'.
/usr/lib/python3.10/threading.py:1018: RuntimeWarning: coroutine 'Server.serve' was never awaited
  self._

KeyboardInterrupt: 

# Final Exercise

In [ ]:
"""
Use the data provided in "transactions.csv" to create a budgeting agent.

Your agent should use the following tools:
- Retrieval from "transactions.csv"
- Calculator

Answer the following questions:

What was my income in December?
What’s the address for the restaurant I went to on December 12th?
I live in San Francisco. Do you think I’m paying too much for rent?
"""


--2024-01-11 20:01:44--  https://drive.google.com/uc?export=download&id=1-jD-CO7UmHG_RsPuNEMTopqqcfmKUr87
Resolving drive.google.com (drive.google.com)... 142.250.98.100, 142.250.98.101, 142.250.98.102, ...
Connecting to drive.google.com (drive.google.com)|142.250.98.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1-jD-CO7UmHG_RsPuNEMTopqqcfmKUr87&export=download [following]
--2024-01-11 20:01:44--  https://drive.usercontent.google.com/download?id=1-jD-CO7UmHG_RsPuNEMTopqqcfmKUr87&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.217.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1938 (1.9K) [application/octet-stream]
Saving to: ‘transactions.csv’

transactions.csv    100%[===================>]   1.89K  --.-KB/

[Document(page_content='Date: 2023-12-01\nDescription: Monthly Salary\nAmount: 3000\nCategory: Income\nSubcategory: Salary', metadata={'row': 25, 'source': 'transactions.csv'}),
 Document(page_content='Date: 2023-12-18\nDescription: Monthly Salary\nAmount: 3000\nCategory: Income\nSubcategory: Salary', metadata={'row': 32, 'source': 'transactions.csv'}),
 Document(page_content='Date: 2023-11-20\nDescription: Monthly Salary\nAmount: 3000\nCategory: Income\nSubcategory: Salary', metadata={'row': 20, 'source': 'transactions.csv'}),
 Document(page_content='Date: 2023-11-01\nDescription: Monthly Salary\nAmount: 3000\nCategory: Income\nSubcategory: Salary', metadata={'row': 13, 'source': 'transactions.csv'})]